In [3]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown , display
from openai import OpenAI

In [6]:
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")


openai = OpenAI()

In [7]:
class Website:
    url: str
    title: str
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [8]:
ed = Website("https://edwarddonner.com")
print(ed.title)
print(ed.text)

Home - Edward Donner
Home
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and tons of press coverage.
Connect
with me for

In [9]:
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [12]:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
    please provide a short summary of this website in markdown. \
    If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

print(user_prompt_for(ed))

You are looking at a website titled Home - Edward Donner
The contents of this website is as follows;     please provide a short summary of this website in markdown.     If it includes news or announcements, then summarize these too.

Home
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired i

In [15]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "system", "content": user_prompt_for(website)}
    ]

In [16]:
messages_for(ed)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'system',
  'content': 'You are looking at a website titled Home - Edward Donner\nThe contents of this website is as follows;     please provide a short summary of this website in markdown.     If it includes news or announcements, then summarize these too.\n\nHome\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (\nvery\namateur) and losing myself in\nHacker News\n, nodding my head sagely to things I only half understand.\nI’m the co-founder and CTO of\nNebula.io\n. We’re applying AI to a field where it can make a mas

In [21]:
def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

summarize("https://edwarddonner.com")

"# Edward Donner's Website Summary\n\nEdward Donner's website serves as a platform for sharing his passion for coding and experimenting with Large Language Models (LLMs). He is the co-founder and CTO of Nebula.io, where the focus is on leveraging AI to assist in talent discovery and management. His startup experiences include founding untapt, which was acquired in 2021.\n\n### Key Features:\n- **Outsmart**: An interactive arena designed for LLMs to engage in a competitive setting that tests their diplomatic and strategic skills.\n- **Professional Background**: Information about Edward's roles in the tech industry, including his endeavors in AI and talent management.\n- **Posts**: \n\n### Recent Announcements/Posts:\n1. **December 21, 2024**: Welcome message for SuperDataScientists.\n2. **November 13, 2024**: Resources for mastering AI and LLM engineering.\n3. **October 16, 2024**: Guidance for transitioning from software engineer to AI data scientist.\n4. **August 6, 2024**: Introducti

In [22]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

display_summary("https://edwarddonner.com")

# Summary of Edward Donner's Website

Edward Donner's website showcases his interests in AI, specifically in the experimentation with large language models (LLMs). He is the co-founder and CTO of Nebula.io, a company focused on utilizing AI to enhance talent acquisition and management. Previously, he founded the AI startup untapt, which was acquired in 2021.

## Key Features:
- **Outsmart Arena**: A unique platform where LLMs compete in games of diplomacy and strategy.
- **Personal Interests**: In addition to coding and AI, he enjoys DJing and electronic music production.
- **Professional Background**: Experience in founding and leading AI-focused startups with a patented matching model and a reputable platform.

## Recent Announcements:
- **December 21, 2024**: Welcoming the audience of SuperDataScientists.
- **November 13, 2024**: Sharing resources for mastering AI and LLM engineering.
- **October 16, 2024**: Providing resources for transitioning from a software engineer to an AI data scientist.
- **August 6, 2024**: Introducing the Outsmart LLM Arena concept. 

The site encourages connections and engagement through various platforms.

In [23]:
display_summary("https://cnn.com")

# Summary of CNN

CNN is a major news outlet providing extensive coverage of breaking news, current events, and in-depth analysis across various subjects including politics, business, health, entertainment, and global affairs. It covers live updates on significant occurrences such as wildfires, international conflicts, and political developments.

## Notable News Highlights:
- **California Wildfires**: Firefighters are actively combating the Palisades Fire, which is threatening areas in West LA. Residents are facing tough decisions about evacuation and what belongings to take.
- **Protests in China**: Recent demonstrations have escalated into violence following the death of a student, indicating rising unrest in the region.
- **Ukraine Conflict**: Reports confirm the capture of North Korean soldiers fighting alongside Russian forces.
- **Israel-Hamas Talks**: There are indications that discussions regarding a ceasefire-for-hostages agreement may be progressing, with Israel sending a Mossad chief to Qatar.
- **Health Reports**: Concerns about lead and cadmium in muscle-building protein powders have been highlighted.

CNN also features multimedia content including videos and detailed analyses of current topics, along with sections on climate, sports, technology, and lifestyle.

In [24]:
display_summary("https://point.md")

# Summary of the Point.md Website

**Point.md** is a news website focused on providing updates and analysis on various topics related to Moldova, including politics, economics, society, incidents, international news, sports, lifestyle, automotive, science, and technology.

## Recent News Highlights
- **Political Statements**: Moldovan Deputy Prime Minister and Foreign Minister Mihai Popșoi indicated that the country's European integration and the withdrawal of Russian troops will positively affect the resolution process in Transnistria.
- **Call for Dialogue**: Former Prime Minister Vlad Filat urged current Prime Minister Maia Sandu to initiate direct dialogue with Tiraspol regarding Transnistria.
- **Energy Crisis Impact**: Companies in Transnistria have shifted to a night work schedule due to the ongoing energy crisis.
- **Legal Actions**: The head of a Moldindconbank branch has been sentenced to prison.
- **Import/Export Regulations**: Authorities remind citizens about the requirements for importing and exporting vehicles.
- **Border Incidents**: Ukrainian border guards apprehended 11 men at the border with Moldova who were allegedly paying up to $10,000.

This summary encapsulates the recent developments relevant to Moldova and reflects ongoing conversations regarding governance, economic challenges, and cross-border issues.